In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path

data_root = 'data'

def generate_moves(fast_move_json, charged_move_json, outfile):
	fast_moves_df = pd.read_json(f'{data_root}/pogoapi/{fast_move_json}')
	charged_moves_df = pd.read_json(f'{data_root}/pogoapi/{charged_move_json}')

	fast_moves_df.rename(columns={
		'name':'fast_name',
		'type': 'fast_type',
		'power': 'fast_power',
		'energy_delta': 'fast_energy'
	}, inplace=True)

	charged_moves_df.rename(columns={
		'name':'charged_name',
		'type': 'charged_type',
		'power': 'charged_power',
		'energy_delta': 'charged_energy'
	}, inplace=True)

	fast_moves_df = fast_moves_df[['fast_name', 'fast_type', 'fast_power', 'fast_energy']]
	charged_moves_df = charged_moves_df[['charged_name', 'charged_type', 'charged_power', 'charged_energy']]

	# Generate power per energy
	fast_moves_df['fast_power_per_energy'] = fast_moves_df['fast_power'] / fast_moves_df['fast_energy']
	charged_moves_df['charged_power_per_energy'] = charged_moves_df['charged_power'] / charged_moves_df['charged_energy']

	# Merging moves together
	merged_moves_df = pd.merge(fast_moves_df, charged_moves_df, how='cross')

	# Generate charged rate
	merged_moves_df['charged_rate'] = np.ceil(merged_moves_df['charged_energy'].abs() / merged_moves_df['fast_energy'])

	# Handle infinity values
	merged_moves_df['charged_rate'] = merged_moves_df['charged_rate'].replace([np.inf, -np.inf], None)
	merged_moves_df['fast_power_per_energy'] = merged_moves_df['fast_power_per_energy'].replace([np.inf, -np.inf], None)
	merged_moves_df['charged_power_per_energy'] = merged_moves_df['charged_power_per_energy'].replace([np.inf, -np.inf], None)

	# Sorting
	merged_moves_df.sort_values(by=['charged_rate'], ascending=True, inplace=True)

	# Create parent directories
	Path(f'{data_root}/csv').mkdir(parents=True, exist_ok=True)
	Path(f'{data_root}/json').mkdir(parents=True, exist_ok=True)

	# Writing to file
	merged_moves_df.to_csv(f'{data_root}/csv/{outfile}.csv',index=False)
	merged_moves_df.to_json(f'{data_root}/json/{outfile}.json', orient='records')

	# Output results
	# display(merged_moves_df)
	print(f'{outfile} moves generated')

generate_moves('fast_moves.json', 'charged_moves.json', 'gym_raid_moves')
generate_moves('pvp_fast_moves.json', 'pvp_charged_moves.json', 'pvp_moves')